In [ ]:
from First_Dataset import First_Dataset

In [5]:
dataset = First_Dataset()
dataset.preprocess()

In [8]:
from sklearn import preprocessing

total_sentences = list(dataset.df['tweet'].values)
total_labels = list(dataset.df['class'].values)
le = preprocessing.LabelEncoder()
le.fit(total_labels)
encoded_labels = le.transform(total_labels)

In [9]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_length = 0
for sentence in total_sentences:
    #print(sentence)
    length = len(tokenizer.tokenize(sentence))
    if length > max_length:
        max_length  = length
print("max token length is: ",max_length)

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<?, ?B/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 486kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.22MB/s]
config.json: 100%|██████████| 570/570 [00:00<?, ?B/s] 


max token length is:  159


In [10]:
from Data_Preprocessing import *

X_train, X_val, X_test, Y_train, Y_val, Y_test = split_data(total_sentences,total_labels,0.1,0.1)

In [12]:
import torch

def encoder_generator(sentences,labels):
    
    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):
        
        sent_index.append(index)
        
        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=160,
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0)
    attention_masks = torch.cat(attention_masks,dim=0)
    labels = torch.tensor(labels)
    sent_index = torch.tensor(sent_index)

    return sent_index,input_ids,attention_masks,labels

sent_index,input_ids,attention_masks,encoded_label_tensors = encoder_generator(X_train,Y_train)
val_sent_index,val_input_ids,val_attention_masks,encoded_val_label_tensors = encoder_generator(X_val,Y_val)
test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors = encoder_generator(X_test,Y_test)


In [14]:
from torch.utils.data import TensorDataset,random_split

train_dataset = TensorDataset(input_ids,attention_masks,encoded_label_tensors)
val_dataset = TensorDataset(val_sent_index,val_input_ids,val_attention_masks,encoded_val_label_tensors)
test_dataset = TensorDataset(test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors)


In [15]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=8

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(val_dataset,
                              sampler=SequentialSampler(val_dataset),
                              batch_size=bs)
test_data_loader = DataLoader(test_dataset,
                            sampler=SequentialSampler(test_dataset),
                            batch_size=bs)

In [16]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                     num_labels=len(le.classes_),
                                                     output_attentions=False,
                                                     output_hidden_states=False,
                                                     )

model.safetensors: 100%|██████████| 440M/440M [04:12<00:00, 1.75MB/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import tensorflow as tf

output = model(input_ids, attention_mask=attention_masks)[0]
model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    {'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask']},
    {'output_1': train_labels},
    epochs=3,
    batch_size=32,
    validation_data=(
        {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']},
        {'output_1': test_labels}
    )
)
